### Deep FBA Trader model creation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.utils import Sequence
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os

Combining training data into one csv file

In [ ]:
root_folder_path = 'C:/Users/camer/Documents/Masters Thesis/Data/Training data/'
specific_folder = '1 sec batch full trading day'
folder_path = root_folder_path + specific_folder

data_sets = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        data_set = pd.read_csv(file_path)
        data_sets.append(data_set)

large_csv = pd.concat(data_sets, ignore_index=True)
large_csv.to_csv(f'{folder_path}/all_data.csv', index=False)
training_data = large_csv
training_data

Fitting the scaler for normalisation

In [2]:
folder_path = 'C:/Users/camer/Documents/Masters Thesis/Data/Training data/1 sec batch full trading day'

training_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]


In [13]:
input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

for file in training_files:
    for chunk in pd.read_csv(file, chunksize=32768):
        X = chunk.drop(columns=['time_of_trade', 'final_trade_price']).values
        y = chunk[['final_trade_price']].values
        input_scaler.partial_fit(X)
        output_scaler.partial_fit(y)

input_scaler_filename = 'input_scaler_equil_predictor_minmax.joblib'
output_scaler_filename = 'output_scaler_equil_predictor_minmax.joblib'
joblib.dump(input_scaler, input_scaler_filename)
joblib.dump(output_scaler, output_scaler_filename)

KeyboardInterrupt: 

In [14]:
input_scaler = joblib.load('scalers/input_scaler_equil_predictor_minmax.joblib')
output_scaler = joblib.load('scalers/output_scaler_equil_predictor_minmax.joblib')

Creating data generator class to batch data for training

In [15]:
class DataGenerator(Sequence):
    def __init__(self, training_files, batch_size=1024, input_scaler=None, output_scaler=None, shuffle=False):
        self.training_files = training_files
        self.batch_size = batch_size
        self.input_scaler = input_scaler
        self.output_scaler = output_scaler
        self.shuffle = shuffle
        self.current_file_idx = 0
        self.load_next_file()
        
    def load_next_file(self):
        if self.current_file_idx >= len(self.training_files):
            self.current_file_idx = 0
        self.current_file = self.training_files[self.current_file_idx]
        self.data_iterator = pd.read_csv(self.current_file, chunksize=self.batch_size)

    def __len__(self):
        total_rows = sum([pd.read_csv(file, usecols=[0]).shape[0] for file in self.training_files])
        return total_rows // self.batch_size

    def __getitem__(self, index):
        try:
            chunk = next(self.data_iterator)
        except StopIteration:
            self.current_file_idx += 1
            self.load_next_file()
            chunk = next(self.data_iterator)

        X_batch = chunk.drop(columns=['time_of_trade', 'final_trade_price']).values
        y_batch = chunk['final_trade_price'].values

        if self.input_scaler:
            X_batch = self.input_scaler.transform(X_batch)
        if self.output_scaler:
            y_batch = self.output_scaler.transform(y_batch.reshape(-1, 1)).flatten()

        X_batch = X_batch.reshape((-1, 1, X_batch.shape[1]))

        return X_batch.astype(np.float16), y_batch.astype(np.float16)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.training_files)
        self.current_file_idx = 0
        self.load_next_file()

In [21]:
train_files, test_files = train_test_split(training_files, test_size=0.1, random_state=42)

train_generator = DataGenerator(train_files, batch_size=1024, input_scaler=input_scaler, output_scaler=output_scaler, shuffle=False)
test_generator = DataGenerator(test_files, batch_size=1024, input_scaler=input_scaler, output_scaler=output_scaler, shuffle=False)

# test_data_list = []

# for test_file in test_files:
#     test_data_sample = pd.read_csv(test_file)
#     test_data_list.append(test_data_sample)
 
# test_data = pd.concat(test_data_list, ignore_index=True)

# X_test = test_data.drop(columns=['time_of_trade', 'final_trade_price']).values
# y_test = test_data['final_trade_price'].values

# X_test_scaled = scaler.transform(X_test)

# X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


In [ ]:
model = Sequential()
model.add(InputLayer(shape=(1, train_generator[0][0].shape[2]), dtype=tf.float16))
model.add(LSTM(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1))


adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(train_generator, epochs=15, validation_data=test_generator, verbose=1)

Epoch 1/15


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45220/45227 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0166

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45227/45227 ━━━━━━━━━━━━━━━━━━━━ 649s 13ms/step - loss: 0.0166 - val_loss: 0.0142
Epoch 2/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 574s 12ms/step - loss: 0.0113 - val_loss: 0.0093
Epoch 3/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 632s 12ms/step - loss: 0.0092 - val_loss: 0.0093
Epoch 4/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 683s 13ms/step - loss: 0.0091 - val_loss: 0.0089
Epoch 5/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 603s 12ms/step - loss: 0.0089 - val_loss: 0.0087
Epoch 6/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 649s 13ms/step - loss: 0.0089 - val_loss: 0.0086
Epoch 7/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 566s 11ms/step - loss: 0.0088 - val_loss: 0.0087
Epoch 8/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 647s 13ms/step - loss: 0.0088 - val_loss: 0.0086
Epoch 9/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 590s 11ms/step - loss: 0.0088 - val_loss: 0.0086
Epoch 10/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 652s 13ms/step - loss: 0.0088 - val_loss: 0.0086
Epoch 11/15
45227/45227 ━━━━━━━━━━━━━━━━━━━━ 589s 11ms/step - loss: 0.0088 - val

In [ ]:
model.save('Neural_network_models/1secbatch_model_v10_MINMAX.keras')
joblib.dump(history.history, 'training_history_equil_pred_v10.pkl')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## --------------- Old code below ----------------

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, training_files, batch_size=1024, scaler=None, shuffle=False):
        self.training_files = training_files
        self.batch_size = batch_size
        self.scaler = scaler
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.training_files))
        self.current_file = None
        self.current_file_idx = 0
        self.data_iterator = None
        self.on_epoch_end()

    def __len__(self):
        total_rows = sum([pd.read_csv(file, usecols=[0]).shape[0] for file in self.training_files])
        print(total_rows)
        return total_rows // self.batch_size

    def __getitem__(self, index):
        if self.data_iterator is None or self.current_file is None or not self.has_next_chunk():
            self.current_file = self.training_files[self.current_file_idx]
            self.data_iterator = pd.read_csv(self.current_file, chunksize=self.batch_size)
            self.current_file_idx = (self.current_file_idx + 1) % len(self.training_files)

        chunk = next(self.data_iterator)
        X_batch = chunk.drop(columns=['time_of_trade', 'final_trade_price']).values
        y_batch = chunk['final_trade_price'].values

        if self.scaler:
            X_batch = self.scaler.transform(X_batch)

        X_batch = X_batch.reshape((-1, 1, X_batch.shape[1]))

        return X_batch.astype(np.float16), y_batch
    
    def on_epoch_end(self):
        self.current_file_idx = 0
        self.indexes = np.arange(len(self.training_files))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        self.current_file = None
        self.data_iterator = None


    def has_next_chunk(self):
        try:
            _ = next(self.data_iterator)
            self.data_iterator = pd.read_csv(self.current_file, chunksize=self.batch_size)
            return True
        except StopIteration:
            return False


Prepping the training data for input - No longer in use

In [4]:
X = training_data.drop(columns=['final_trade_price', 'time_of_trade'])
y = training_data['final_trade_price']

X = X.values
y = y.values

X = X.reshape((X.shape[0], 1, X.shape[1]))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_2d = X_train.reshape((X_train.shape[0], X_train.shape[2]))
X_test_2d = X_test.reshape((X_test.shape[0], X_test.shape[2]))

X_train_scaled = scaler.fit_transform(X_train_2d)
X_test_scaled = scaler.transform(X_test_2d)

scaler_filename = 'scaler.joblib'
joblib.dump(scaler, scaler_filename)

X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

X_train_scaled = X_train_scaled.astype(np.float16)
X_test_scaled = X_test_scaled.astype(np.float16)

Building the model

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), dtype=tf.float16))
model.add(LSTM(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, verbose=1)
test_loss = model.evaluate(X_test_scaled, y_test, verbose=1)

Save model

In [ ]:
model.save('Neural_network_models/1secbatch_model_v6_current_best.keras')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()